## 记忆增强检索的示例

本节我们将深入探讨如何通过集成记忆组件，增强我们的数据连接模块LEDVR工作流中的数据检索能力，从而提升QA问答应用的回答质量。我们将以《链》章节中的代码案例为基础，介绍如何在已有的检索器基础上，添加记忆组件，使ConversationalRetrievalChain链组件具有“记忆”能力。

在此基础上，我们的案例应用将得以进一步提升。以LUA语言开发者在线文档问答为例，通过增强后的链组件，我们的程序不仅能在 http://developers.mini1.cn/wiki/luawh.html 中找到相关的文档内容，准确回答如“LUA宿主语言是什么”等问题，更能够记住用户在对话中的相关信息，如用户的名字，从而对更多复杂的问题提供更准确的答复。例如，当用户在问完几轮关于LUA语言编程的问题之后，如果他们再次提到他们的名字或引用他们之前提到的信息，增强后的ConversationalRetrievalChain链组件能够记住并理解这些信息，提供更加精确和个性化的回答。

希望读者能够理解并掌握如何将记忆组件与链组件集成，从而实现数据检索能力的增强，提升QA问答应用的回答质量。同时，我们也将向读者展示如何通过增加记忆组件，使我们的程序更具人性化，能够更好地满足用户的需求。使用加载器开始，到 ConversationalRetrievalChain 的运行都与《链》章节一样。如果你已经很熟悉这段代码，可以直接跳到 ConversationalRetrievalChain 运行代码之后阅读。



首先，我们需要从网络加载文档。这可以通过使用WebBaseLoader来完成：



In [2]:
from langchain.document_loaders import WebBaseLoader
openai_api_key=""
loader = WebBaseLoader("http://developers.mini1.cn/wiki/luawh.html")
data = loader.load()



你也可以选择其他的加载器和其他的文档资源。接下来，我们需要创建一个嵌入模型实例的包装器，这可以使用OpenAIEmbeddings完成：

In [3]:


from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)



然后，我们需要将文档切割成块，这可以通过使用RecursiveCharacterTextSplitter来完成：



In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)



接着，我们需要创建一个向量存储库，这里我们选择使用FAISS：



In [5]:
from langchain.vectorstores import FAISS
vectordb = FAISS.from_documents(documents=splits,embedding=embedding)



现在我们有了一个向量存储库，我们可以用它来创建一个检索器 retriever，LEDVR 工作流就结束了。整个检索器便是LEDVR工作流的“胜利果实”：



In [6]:
retriever = vectordb.as_retriever()

从这里开始，我们可以将检索器加入到链中。首先，我们需要创建一个LLM 模型包装器，并通过ConversationalRetrievalChain.from_llm方法创建一个ConversationalRetrievalChain链组件的实例：

In [7]:

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
llm = ChatOpenAI(openai_api_key=openai_api_key,temperature=0, model="gpt-3.5-turbo-0613")
qa = ConversationalRetrievalChain.from_llm(llm, retriever)


至此，我们已经完成了一个相当复杂的链 ConversationalRetrievalChain。这个链是我们目前了解的最复杂的链组件，它承担了会话和检索文档的两个重要职责。通过这个链，我们可以用如下的方式来查询问题：

In [8]:
query = "LUA的宿主语言是什么？"
result = qa({"question": query,"chat_history":[]})
result["answer"]

'LUA的宿主语言通常是C或C++。'

#### 记忆增强检索的代码开始

先导入我们的 ConversationBufferMemory 类，这个类实例化的组件时最常见的组件，组件的工作方式也非常简单，仅仅是将所有的聊天记录保存起来，并没有使用算法做截取或者提炼压缩。在提示词模板中可以看到所有的聊天记录。

In [9]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)

我们看看初始化后，记忆组件里保存着什么记录。因为我们刚刚初始化，所以是[]。

In [10]:
memory.chat_memory.messages

[]

我们使用 add_user_message 添加一个人类的消息，向程序介绍我的名字。

In [11]:
memory.chat_memory.add_user_message("我是李特丽")

第一次打印 memory.chat_memory.messages 时为[], 当我推送一条自我介绍后，我们可以看到添加了一条 HumanMessage 消息。

In [12]:
memory.chat_memory.messages

[HumanMessage(content='我是李特丽', additional_kwargs={}, example=False)]

In [13]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='我是李特丽', additional_kwargs={}, example=False)]}

我们采用最直接的方式来演示记忆组件是如何与链组件共同工作的。从提示词模板出发，逐步增加组件的方式，展示这个工作机制。这一次我们使用的链组件是 load_qa_chain。这个链组件的优势在于专门用于QA问答，对于合并文档链组件不必掌握这些链的使用方法，只要在这个链上指定类型。

In [14]:
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

使用向量存储库实例 的 similarity_search 方法，测试是否可以检索到与问题相关的文档了。我们可以打印 len(docs)， 看看这个问题搜索到了几个文档片段。

In [22]:
query = "LUA 的宿主语言是什么？"
docs = vectordb.similarity_search(query)
len(docs)

4

打印 docs 显示：

In [23]:
docs

[Document(page_content='Lua提供了非常易于使用的扩展接口和机制：由宿主语言(通常是C或C++)提供这些功能，Lua可以使用它们，就像是本来就内置的功能一样。 其它特性:', metadata={'source': 'http://developers.mini1.cn/wiki/luawh.html', 'title': '什么是Lua编程 | 开发者脚本帮助文档', 'description': '迷你世界开发者接口文档', 'language': 'zh-CN'}),
 Document(page_content='快速上手  开发者脚本怎么使用如何编辑脚本基础入门  什么是Lua编程Lua基本语法Lua数据类型Lua的变量Lua的循环语句流程控制语句（条件语句）Lua函数Lua运算符Lua字符串Lua数组Lua table(表)基础脚本示例及解读基础学习案例中级开发者  脚本是怎么运行的？迷你世界API接口  学习案例高手的世界  游戏数据类型小游戏开发《争分夺秒》小游戏开发《坦克对战》迷你世界ID查询  迷你世界ID查询脚本常见问题  开发者常见问题更新记录  更新日志   # 什么是Lua编程 以下部分内容引用自：https://www.runoob.com/lua/ # lua语言简介 概述 Lua是一种轻量小巧的脚本语言，用标准C语言编写并以源代码形式开放， 其设计目的是为了嵌入应用程序中，从而为应用程序提供灵活的扩展和定制功能。 特点 轻量级、可扩展，等等 轻量级: 它用标准C语言编写并以源代码形式开放，编译后仅仅一百余K，可以很方便的嵌入别的程序里。 可扩展:', metadata={'source': 'http://developers.mini1.cn/wiki/luawh.html', 'title': '什么是Lua编程 | 开发者脚本帮助文档', 'description': '迷你世界开发者接口文档', 'language': 'zh-CN'}),
 Document(page_content='什么是Lua编程 | 开发者脚本帮助文档\n\n\n\n\n\n\n\n  开发者脚本帮助文档   \n  API接口查询\n\n  迷你世界官网\n   \n  星启计划\n      \n  API接口查询', 

使用提示模板包装器，我们自定义一个提示词模板字符串。提示词内容四部分：一是对模型的指导词：“请你回答问题的时候，请依据文档内容和聊天记录回答，如果在其中找不到相关信息或者答案，请回答不知道。” ； 二是用问题检索到的相关文档内容：“文档内容是：{context}” ；三是记忆组件输出的记忆内容：“聊天记录是：{chat_history}” ；四是用户的输入： “Human: {human_input}”。

In [16]:
template = """你是说中文的chatbot.

请你回答问题的时候，请依据文档内容和聊天记录回答，如果在其中找不到相关信息或者答案，请回答不知道。

文档内容是：{context}

聊天记录是：{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
chain = load_qa_chain(
    llm=llm, chain_type="stuff", memory=memory, prompt=prompt
)

In [17]:
query = "LUA 的宿主语言是什么？"
docs = vectordb.similarity_search(query)
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-VEnNKTtyYXEBdaVF7u4kNojB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-VEnNKTtyYXEBdaVF7u4kNojB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/

{'output_text': 'Lua的宿主语言通常是C或C++。'}

In [18]:
query = "我的名字是李特丽。你叫什么？"
docs = vectordb.similarity_search(query)
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-VEnNKTtyYXEBdaVF7u4kNojB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-VEnNKTtyYXEBdaVF7u4kNojB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/

{'output_text': '我是一个中文的chatbot。'}

In [19]:
query = "LUA 的循环语句是什么？"
docs = vectordb.similarity_search(query)
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-VEnNKTtyYXEBdaVF7u4kNojB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-VEnNKTtyYXEBdaVF7u4kNojB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/

{'output_text': 'Lua的循环语句有while循环、for循环和repeat...until循环。'}

In [20]:
query = "我的名字是什么？"
docs = vectordb.similarity_search(query)
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-VEnNKTtyYXEBdaVF7u4kNojB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-VEnNKTtyYXEBdaVF7u4kNojB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/

{'output_text': '你的名字是李特丽。'}

In [21]:
print(chain.memory.buffer)

Human: LUA 的宿主语言是什么？
AI: Lua的宿主语言通常是C或C++。
Human: 我的名字是李特丽。你叫什么？
AI: 我是一个中文的chatbot。
Human: LUA 的循环语句是什么？
AI: Lua的循环语句有while循环、for循环和repeat...until循环。
Human: 我的名字是什么？
AI: 你的名字是李特丽。


In [25]:
query = "我的名字是什么？"
docs = vectordb.similarity_search(query)
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': '你的名字是李特丽。'}